In [ ]:
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
from matplotlib.patches import Polygon, Rectangle
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from casadi import *
import calendar
import casadi as cd
from sklearn.linear_model import LinearRegression
import matplotlib.dates as mdates

Reading the bill data and the power data

In [ ]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)
    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)
            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
            # Append the dataframe to the list
            df_list.append(df)

bill_path = 'D:\\mlinternship\\IITGuwahatiElectricityBills'
for filename in os.listdir(bill_path):
    # Check if the file has the '.xlsx' extension
    if filename.endswith('.xlsx'):
        file_path = os.path.join(bill_path, filename)
        # Use the custom date parser function
        bill_df = pd.read_excel(file_path)

bill_df['Month'] = pd.to_datetime(bill_df['Month'])
#I assumed its in kilowatts
bill_df['MW'] = bill_df['Number of units of electricity consumed']/1000
bill_df.drop(['Number of units of electricity consumed'], axis=1)
bill_df

Reading the temperature data and merging the power and temperature data

In [ ]:
#read the power data
power_df = pd.concat(df_list, ignore_index=True)
power_df.to_csv('power_datacsv.csv')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
power_df['MW'] = power_df['MW'].replace('nr', np.nan)
power_df = power_df[['Time', 'MW']]
power_df['MW'] = power_df['MW'].astype(str)
power_df['MW'] = pd.to_numeric(power_df['MW'].str.replace(',', '.'), errors='coerce')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df = power_df.sort_values('Time')
full_power_df = power_df.copy()

# read the temperature data csv
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])
temperature_df['Time'] = pd.DatetimeIndex(temperature_df['Time']) + timedelta(hours=5,minutes=30)
temperature_df['temperature'] = pd.to_numeric(temperature_df['temperature'], errors='coerce')
temperature_df.set_index('Time', inplace=True)
temperature_df['temperature'] = temperature_df['temperature'].interpolate(method='polynomial', order = 5)
temperature_df.reset_index(inplace=True)

# joining the two dataframes such that the temperature data is only taken if there exists a reading in the power data dataframe
df = pd.merge(power_df, temperature_df, on='Time', how='left')
df['temperature'] = df['temperature'].interpolate(method='polynomial', order = 5)
df['Time'] = pd.to_datetime(df['Time'])

full_model_start_time = pd.Timestamp('2022-02-07 00:00:00')
full_model_end_time = pd.Timestamp('2022-12-25 23:00:00')
df = df[(df['Time'] >= full_model_start_time) & (df['Time'] <= full_model_end_time)]
df = df.drop(df[df['MW'] > 20].index)
df = df.sort_values('Time')
df.reset_index(drop=True)
df

Creating the CDH and Omega columns

In [ ]:
TcoolStPt = 31
CDH = df['temperature'] - TcoolStPt
CDH.clip(lower=0, inplace=True)
CDH = pd.DataFrame(data=CDH.values, columns=['CDH'], index=df.index)
# Concatenate CDH with the original DataFrame using the index
df = pd.concat([df, CDH], axis=1)
df = df.sort_values('Time')
df.reset_index(inplace=True, drop = True)

numOmegas = 24 * 7
num_of_rows = df.shape[0]
omegas = np.zeros((num_of_rows, numOmegas))  # Assuming numOmegas columns for omegas
concatenated_data = np.concatenate((df, omegas), axis=1)
column_names = ['Time', 'MW', 'temperature', 'CDH']
for i in range(1, numOmegas + 1,1):
    column_names.append('omega' + str(i))

df = pd.DataFrame(concatenated_data, columns=column_names)
df['Time'] = pd.to_datetime(df['Time'])
for i in range(0,num_of_rows):
        datetime = df.Time.loc[i]
        hourOfWeekIndex = int(datetime.dayofweek*24+(datetime.hour+1))
        x = np.zeros((1,numOmegas))
        x[0,hourOfWeekIndex-1]=1
        omegas[i,:]=x

df.iloc[:,4:]=omegas
DF = df.copy()
df = df.dropna()
df.reset_index(inplace=True, drop = True)
df

In [ ]:
'''
start_time = pd.Timestamp('2022-07-04 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)
df = df[training_mask]'''

doing the optimization in one shot

In [ ]:
#tried to do the temperature and behavior part together
start_time = pd.Timestamp('2022-08-01 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')

training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)
training_df = df[training_mask]
d = training_df.loc[:,'MW']
phi_temperature = training_df.loc[:,'CDH']
phi_behavior = training_df.loc[:,'omega1':'omega168']
opti = cd.Opti()
temperature_theta = opti.variable()
behavior_theta = opti.variable(168)
total_sum =0
for i in range (0,len(d)):
    phi_behavior_i = (cd.MX(phi_behavior.iloc[i].values))
    phi_behavior_i =  cd.vertcat(phi_behavior_i)
    residual = (d.iloc[i] - ((phi_temperature.iloc[i] * temperature_theta) + cd.dot(phi_behavior_i, behavior_theta)))**2
    total_sum += residual

opti.subject_to(temperature_theta >= 0)
e1 = 3.0
e2 = 3.0

for i in range(0, 167):
    opti.subject_to((behavior_theta[i + 1] - behavior_theta[i]) <= e1)
    opti.subject_to((behavior_theta[i + 1] - behavior_theta[i]) >= -e1)
    opti.subject_to(behavior_theta[i] >= 0)
    opti.subject_to(behavior_theta[i] <= 7)

for i in range(1, 167):
    opti.subject_to((behavior_theta[i + 1] + behavior_theta[i - 1] - 2 * behavior_theta[i]) <= e2)
    opti.subject_to((behavior_theta[i + 1] + behavior_theta[i - 1] - 2 * behavior_theta[i]) >= -e2)

#this bill part is kind of hardcoded rn
for i in range(7,8):
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_weeks = num_hours / 168.0
    weekly_behavior_energy_use = 0
    temperature_energy_use = 0
    for j in range(len(training_df)):
        temperature_energy_use += temperature_theta * training_df.iloc[j]['CDH']

    for j in range(0, 168):
        weekly_behavior_energy_use += behavior_theta[j]
    weekly_energy_use = weekly_behavior_energy_use +  temperature_energy_use
    opti.subject_to(weekly_energy_use < 1.5 * bill_df.iloc[i]['MW'] / num_weeks)
    opti.subject_to(weekly_energy_use > 0.5 * bill_df.iloc[i]['MW'] / num_weeks)

# Solve the optimization problem
opti.minimize(total_sum)
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()

optimal_temperature_theta = sol.value(temperature_theta)
optimal_behavior_theta = sol.value(behavior_theta)
print("Optimal value of temperature_theta:", optimal_temperature_theta)
print("Optimal values of theta:", optimal_behavior_theta)


In [ ]:
#final model with df(even null values)
training_df = df[training_mask]
d = training_df['MW']
c = np.concatenate((np.array([optimal_temperature_theta]),optimal_behavior_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
full_model = LinearRegression(fit_intercept=False)
full_model.fit(phi, d)
full_model.coef_ = c
full_model.intercept_ = 0
yhat = full_model.predict(phi.values)
full_modelScore = full_model.score(phi,d)

print ('score for constructed full model on full data: ', full_modelScore)
fig,(ax2) = plt.subplots(nrows=1,figsize=(10,9))
_=ax2.plot(training_df['Time'],d,label='meas', marker = 'o')
_=ax2.plot(training_df['Time'],yhat,label='pred:', marker = 'x')
ax2.set_title('measured vs predicted data (full constructed model) (only for non null values)')
_=ax2.legend()
plt.show()

actual vs predicted data with graph shading

In [ ]:
#graph with shaded portion to represent missing values

#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
#actual_data = training_df['MW'].dropna()
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
d = training_df['MW']
c = np.concatenate((np.array([optimal_temperature_theta]),optimal_behavior_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace = True)

fig10,(ax10) = plt.subplots(nrows=1,figsize=(10,9))
first_time = True
j = 0

for i in range(1, len(training_df)):
    if (j < len(actual_df)) and (training_df.iloc[i]['Time'] == actual_df.iloc[j]['Time']):
        j+=1
    else:
        iterator = i
        last_iterator = i-1
        first_time= False
        start =  mdates.date2num(training_df.iloc[last_iterator]['Time'])
        end =  mdates.date2num(training_df.iloc[iterator]['Time'])
        print(f'index: {i} start:{start}, width:{end-start}')
        rect = Rectangle((start, 0), end-start, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)
        ax10.add_patch(rect)

locator = mdates.AutoDateLocator(minticks=3)
formatter = mdates.AutoDateFormatter(locator)
ax10.xaxis.set_major_locator(locator)
ax10.xaxis.set_major_formatter(formatter)
_=ax10.plot(actual_df['Time'][training_mask],actual_df[training_mask]['MW'],label='meas', marker = 'o', color = 'r', zorder = 5)
_=ax10.plot(training_df['Time'][training_mask],yhat,label='pred:', marker = 'x', color = 'y', zorder = 5)
ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()


In [ ]:
stop

In [ ]:
#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
#actual_data = training_df['MW'].dropna()
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
d = training_df['MW']
c = np.concatenate((np.array([optimal_temperature_theta]),optimal_behavior_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace = True)



fig10,(ax10) = plt.subplots(nrows=1,figsize=(10,9))
first_time = True
for i in range(0, len(actual_df)):
    if training_df.iloc[iterator+1]['Time'] == actual_df.iloc[i]['Time']:
        pass
    else:
        iterator = i
        while (training_df.iloc[iterator]['Time']!= actual_df.iloc[i]['Time']):
            iterator+=1
            if (iterator > len(actual_df)-1):
                iterator-=1
                break
        if first_time == True:
            last_iterator = i
            first_time= False

        start = training_df.iloc[last_iterator]['Time']
        width = training_df.iloc[iterator]['Time'] - training_df.iloc[last_iterator]['Time']
        rect = Rectangle((start, 0), width, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)
        ax10.add_patch(rect)
        last_iterator = iterator

_=ax10.plot(actual_df['Time'][training_mask],actual_df[training_mask]['MW'],label='meas', marker = 'o', color = 'r', zorder = 5)
_=ax10.plot(training_df['Time'][training_mask],yhat,label='pred:', marker = 'x', color = 'y', zorder = 5)
ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()



In [ ]:
stop

In [ ]:
#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
phi = training_df.loc[:, 'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace=True)

# Reset indices for both training_df and actual_df
training_df.reset_index(drop=True, inplace=True)
actual_df.reset_index(drop=True, inplace=True)
fig10, (ax10) = plt.subplots(nrows=1, figsize=(10, 9))

# Reset indices for consistency
training_df_reset = training_df.reset_index(drop=True)
actual_df_reset = actual_df.reset_index(drop=True)

# Initialize variables to track the start of mismatched regions
start_idx = 0

# Iterate through timestamps and plot rectangles for the mismatched regions
for i in range(len(actual_df_reset)):
    if training_df_reset['Time'][i] != actual_df_reset['Time'][i]:
        # End of mismatched region, plot a rectangle
        end_idx = i
        rect = Rectangle((start_idx, 0), end_idx - start_idx, 10, linewidth=1, facecolor='green', alpha=0.7, zorder=10)
        ax10.add_patch(rect)
        start_idx = i + 1  # Update start index for the next region

# Plot the actual and predicted data
_=ax10.plot(actual_df_reset['Time'], actual_df_reset['MW'], label='meas', marker='o', color='r', zorder=5)
_=ax10.plot(training_df_reset['Time'], yhat, label='pred:', marker='x', color='y', zorder=5)

ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()


In [ ]:
'''
PRCATICE
x = SX.sym("x",3)
f = x[0]**2 + x[1]**2 + x[2]**2
g1 = cd.fabs(x[2] - x[1]) - 1
g2 = cd.fabs(x[1] - x[0]) - 1
g3 = x[0] + x[1] + x[2] - 5
g4 = x[0]
g5 = x[1]
g6 = x[2]
g = cd.vertcat(g1,g2,g3,g4,g5,g6)
nlp = {'x': x, 'f': f, 'g': g}
MySolver = 'ipopt'
opts = {}
# Allocate a solver
solver = nlpsol("solver", MySolver, nlp, opts)
# Solve the NLP
lbg = [0,0,0, 0,0,0]
ubg = [inf, inf,5,inf,inf,inf]
sol = solver(lbg=lbg, ubg=ubg)
# Print solution
print("-----")
print("objective at solution = ", sol["f"])
print("primal solution = ", sol["x"])
print("dual solution (x) = ", sol["lam_x"])
print("dual solution (g) = ", sol["lam_g"])
'''
